# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *"A non-local algorithm for image denoising"* (autorzy: Antoni Buades, Bartomeu Coll i Jean Michel Morel) na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{2}
\end{equation}

gdzie:
\begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{3}
\end{equation}

- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując: jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate
mat = loadmat("MR_data.mat")
Input = mat['I_noisy2']
plt.imshow(Input.astype("uint8"))
plt.axis("off")
plt.gray()
plt.show()

In [ ]:
import timeit
def euclidean_norm(*args):
    return np.sqrt(np.sum([np.power(i,2) for i in args]))

def gauss(x,sigma):
    return (1/(sigma * np.sqrt(2*np.pi)))*np.exp(-(np.power(x,2)/(2*(sigma**2))))

def w(img,x,y,size_v,size_n,sigma,alpha,convo):
    res_t = [[ np.exp(-(
        euclidean_norm(convo*(img[x-size_n//2:x+size_n//2+1,y-size_n//2:y+size_n//2+1] - img[x+i-size_n//2:x+i+size_n//2+1,y+j-size_n//2:y+j+size_n//2+1]))**2)/(alpha*sigma**2))#
    for j in range(-size_v//2,size_v//2)] 
    for i in range(-size_v//2,size_v//2)]
    return np.array(res_t)/np.sum(res_t)

def non_local_means(img, size_v, size_n, sigma, alpha):
    convo = np.array([[gauss(euclidean_norm(i-size_n//2,j-size_n//2),sigma) for j in range(size_n)] for i in range(size_n)])
    res = [[ np.sum(img[i-size_v//2:i+size_v//2+1, j-size_v//2:j+size_v//2+1]*(t:=w(img,i,j,size_v,size_n, sigma,alpha,convo)))
    for j in range(size_v//2 + size_n//2+1,len(img[0])-size_v//2-size_n//2)] 
    for i in range(size_v//2 + size_n//2+1, len(img)-size_v//2-size_n//2)]
    return np.array(res).astype('uint8')


In [ ]:
start_time = timeit.default_timer()
Input_non_local = non_local_means(Input.copy(),7,3,4,5)
print(timeit.default_timer() - start_time)

In [ ]:
def euclidean_norm(*args):
    return np.sqrt(sum([i**2 for i in args]))

def gamma_func(x,sigma):
    return np.exp(- (np.power(x,2)/(2*sigma**2)))

def Billatery_convolution(img, size, v, sigma_r, metric = euclidean_norm , show_fillter = False):
    sigma = np.sqrt(v)
    convo = np.array([[gauss(metric(i-size//2,j-size//2),sigma) for j in range(size)] for i in range(size)])
    res = [[np.sum((temp:=img[i-size//2:i+size//2+1, j-size//2:j+size//2+1])*(b:=convo*gamma_func(metric(img[i,j]*np.ones(temp.shape) - temp),sigma_r)))/np.sum(b) for j in range(size//2,len(img[0])-size//2)] for i in range(size//2, len(img)-size//2)]
    return np.array(res)
start_time = timeit.default_timer()
Input_bill = Billatery_convolution(Input.copy(), 9, 5, 23)
print(timeit.default_timer() - start_time)

In [ ]:
def classic_convolution(img, size, v, metric = euclidean_norm , show_fillter = False):
    sigma = np.sqrt(v)
    convo = np.array([[gauss(metric(i-size//2,j-size//2),sigma) for j in range(size)] for i in range(size)])
    res = [[np.sum(img[i-size//2:i+size//2+1, j-size//2:j+size//2+1]*convo)/np.sum(convo) for j in range(size//2,len(img[0])-size//2)] for i in range(size//2, len(img)-size//2)]
    return np.array(res)
start_time = timeit.default_timer()
Input_classic = classic_convolution(Input.copy(), 9, 5)
print(timeit.default_timer() - start_time)

In [ ]:
fig , ax = plt.subplots(2,2)
fig.set_size_inches(20,20)
ax[1,1].imshow(Input_non_local)
ax[1,1].set_xticks([])
ax[1,1].set_yticks([])
ax[0,1].imshow(Input_classic)
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
ax[1,0].imshow(Input_bill)
ax[1,0].set_xticks([])
ax[1,0].set_yticks([])
ax[0,0].imshow(Input)
ax[0,0].set_xticks([])
ax[0,0].set_yticks([])
ax[0,1].set_title("Gausian blurr")
ax[0,0].set_title("Orginal image")
ax[1,1].set_title("NLM")
ax[1,0].set_title("Bilateral filter")
plt.show()

Najlepszy wynik osiągneła metoda NLM lecz koszt był dużo większy niż pozostałych metod z uwagi na zagnieżdżenie pętli i ciągłego sprawdzania kontekstów punktów. najgorszy wynik dała metoda konwolucji z maską gaussa. uważam że najlepszym wyborem filtracji jest filtracja bilateryjna z uwagi na niewiele większą złożoność obliczeniową i bardzo podobne rezultaty do NLM.